This file contains a utility for chopping up an image of variable size into tiles. Choose the tile
dimensions by trial and error so that they nicely contain a handwritten character. 

In order to get started, upload an image of a journal entry to the colab workspace root directory. Then running the utility will create chopped images in a subdirectory called "cropped." There is a second utility to zip the cropped directory and copy it to your google drive. For this you'll need to authenticate your session through a guid.

First the necessary imports

In [0]:
import os
from PIL import Image
from google.colab import files
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

Utility to crop an image

In [0]:
def crop(path, input, height, width):
    """ 
    Crops input image into height X width tiles  
    
    Parameters: 
    path (string): Path to the directory where you want the output tiles to be 
    created.
    input (string): Name of image file to be chopped
    height (int): height of output tile
    width (int): width of output tile
    """
    im = Image.open(input)
    imgwidth, imgheight = im.size
    
    k = 1
    for i in range(0,imgheight,height):
        for j in range(0,imgwidth,width):
            box = (j, i, j+width, i+height)
            a = im.crop(box)
            a.save(os.path.join(path,"IMG-%s.png" % k))
            k += 1
    print(
        "Successfully chopped {} into {} {}X{} tiles in {} directory".format(
              input, k-1, height, width, path)
    )

Assuming you've uploaded an image file to colab (E.g. "wtf_dec01_2016.png"), let's create a directory called "cropped" to collect the tiles after chopping.

In [0]:
!mkdir cropped

Next we invoke the utility to perform the cropping

In [0]:
crop("cropped/", "wtf_dec01_2016.png", 20, 15)

Next, let's zip the output directory (E.g. "cropped") and copy to your google drive. It will prompt you to authenticate by walking you through your google account and generating a guid that you'll need to paste below to authenticate, so that it can verify your identity to save the file to your google drive.

In [0]:
#@title save data to drive
filename = "wtf_dec01_2016" #@param {type:"string"}
folders_or_files_to_save = "cropped" #@param {type:"string"}
def save_to_google_drive(name, path):
    file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
    }

    media = MediaFileUpload(
          path 
        , mimetype='application/octet-stream'
        , resumable=True
    )

    created = drive_service.files().create(
          body=file_metadata
        , media_body=media
        , fields='id'
    ).execute()

    print('File ID: {}'.format(created.get('id')))

    return created

extension_zip = ".zip"
zip_file = filename + extension_zip
!zip -r $zip_file {folders_or_files_to_save} 
# FOLDERS TO SAVE INTO ZIP FILE

auth.authenticate_user()
drive_service = build('drive', 'v3')

destination_name = zip_file
path_to_file = zip_file
save_to_google_drive(destination_name, path_to_file)

The zip file is not on your google drive for you to uncompress and peruse.